In [1]:
import numpy as np
import pandas as pd

In [2]:
def create_fixture_matrix():
    matrix = np.zeros((2,2))
    np.fill_diagonal(matrix, 2)
    return matrix

def add_fixture():
    pass

def add_player(existing_matrix, isFixture=True):
    # We want to reshape the matrix/array to include an additional player
    n = existing_matrix.shape[0]
    new_matrix = np.insert(existing_matrix, n, 0, axis=1)
    new_row = np.zeros((1, n + 1))
    if isFixture:
        new_row[0][n] = 2
    return np.append(new_matrix, new_row, axis=0)

def calculate_win_scores(group_results_matrix):
    wins = np.array(group_results_matrix).sum(axis=1)
    losses = np.array(group_results_matrix).sum(axis=0)
    win_scores = 1 + ((wins - losses)) / 2
    return win_scores

def check_player(player, player_arr):
    result = np.where(np.array(player_arr) == player)
    return len(result[0])

In [3]:
results = pd.read_csv('colley.csv')
results

,player_1,player_2,score_1,score_2
0,1,2,10,6
1,1,2,10,3
2,1,3,5,10
3,2,3,10,3
4,3,1,7,10
5,2,3,4,10
6,1,4,10,1
7,4,2,10,7


We are going to want to loop through the results 1 at a time and update metrics. 

In [4]:
# Create results and fixture matrices
group_results_matrix = np.zeros((2,2))
fixture_matrix = create_fixture_matrix()

# Initialise the players array using row 1
players = []
players.append(results.iloc[0]['player_1'])
players.append(results.iloc[0]['player_2'])

for i in results.index:
    row = results.iloc[i]
    player_1 = row['player_1']
    player_2 = row['player_2']
    
    # Check to see if a new player has played
    if not check_player(player_1, players):
        players.append(player_1)
        fixture_matrix = add_player(fixture_matrix)
        group_results_matrix = add_player(group_results_matrix, False)
        
    if not check_player(player_2, players):
        players.append(player_2)
        fixture_matrix = add_player(fixture_matrix)
        group_results_matrix = add_player(group_results_matrix, False)
    
    player_1_index = np.where(players == player_1)[0][0]    
    player_2_index = np.where(players == player_2)[0][0]
    
    # Calculate winner here and update results matrix
    if row['score_1'] > row['score_2']:
        group_results_matrix[player_1_index][player_2_index] += 1
    else:
        group_results_matrix[player_2_index][player_1_index] += 1
    
    # Increment total games for both players 
    fixture_matrix[player_1_index][player_1_index] += 1
    fixture_matrix[player_2_index][player_2_index] += 1
    
    # Add the fixture to both players overall
    fixture_matrix[player_1_index][player_2_index] -= 1
    fixture_matrix[player_2_index][player_1_index] -= 1
    
    win_scores = calculate_win_scores(group_results_matrix)
    
    print(np.linalg.solve(fixture_matrix, win_scores))
    

[0.625 0.375]
[0.66666667 0.33333333]
[0.54545455 0.27272727 0.68181818]
[0.57142857 0.42857143 0.5       ]
[0.625  0.4375 0.4375]
[0.625 0.375 0.5  ]
[0.67857143 0.40178571 0.52678571 0.39285714]
[0.66666667 0.33333333 0.5        0.5       ]


In [5]:
players

[1, 2, 3, 4]

In [6]:
fixture_matrix

array([[ 7., -2., -2., -1.],
       [-2.,  7., -2., -1.],
       [-2., -2.,  6.,  0.],
       [-1., -1.,  0.,  4.]])

In [7]:
group_results_matrix

array([[0., 2., 1., 1.],
       [0., 0., 1., 0.],
       [1., 1., 0., 0.],
       [0., 1., 0., 0.]])

In [8]:
wins = np.array(group_results_matrix).sum(axis=1)
losses = np.array(group_results_matrix).sum(axis=0)

In [9]:
wins

array([4., 1., 2., 1.])

In [10]:
losses

array([1., 4., 2., 1.])